In [1]:
DATA_PATH = '../artifacts/data_ingestion/Churn_Modelling.csv'

EXPORT_PATH = '../artifacts/data_preprocessed/1_preprocessed_df.pkl'

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
from sklearn.preprocessing import LabelEncoder

-------

# Processing 

## Preprocess Data 

In [3]:
raw_df = pd.read_csv(DATA_PATH)
df = raw_df.copy()

### 1. Counting Null Values

In [4]:
null_values = df.isnull().sum()
null_values

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### 2. Counting Duplicates

In [5]:
duplicates = df.duplicated().sum()
duplicates

0

### 3. Exited Customer Distribution Analysis

In [6]:
values = df['Exited'].value_counts()
values

Exited
0    7963
1    2037
Name: count, dtype: int64

### 4. Removing Unnecessary Columns

In [7]:
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### 5. Categorizing Dataset Columns (numerical and nominal types)

In [8]:
numbCol = ['EstimatedSalary', 'Balance', 'CreditScore', 'Age']
nomCol =  ['HasCrCard', 'IsActiveMember', 'Geography', 'Gender', 'NumOfProducts', 'Tenure']

[numbCol,nomCol]

[['EstimatedSalary', 'Balance', 'CreditScore', 'Age'],
 ['HasCrCard',
  'IsActiveMember',
  'Geography',
  'Gender',
  'NumOfProducts',
  'Tenure']]

### 6. Outlier Detection Analysis

In [9]:
from scipy import stats

# Define the threshold for outlier detection (the threshold is set to 3 standard deviations from the mean)
threshold = 3

# Create a dictionary to store the count of outliers for each column
outlier_counts = {}

# Iterate over each numerical column
for col in numbCol:

    # Calculate Z-scores for the current column
    z_scores = stats.zscore(df[col])
    
    # Find outliers
    outliers = abs(z_scores) > threshold
    
    # Count the number of outliers for the current column
    outlier_count = outliers.sum()
    
    # Store the count of outliers for the current column in the dictionary
    outlier_counts[col] = outlier_count
    
outlier_counts

{'EstimatedSalary': 0, 'Balance': 0, 'CreditScore': 8, 'Age': 133}

##### Insight:
---------------------
We've found outliers in two columns. Before removing them, we need to ensure that it's okay to drop them. 
Because age can be any number, although we set one limit and check for real outliers. This examination is crucial for maintaining data integrity and reliability in subsequent analyses

### 7. Age Data Integrity Check

In [10]:
# Filter the DataFrame for values in the "Age" column greater than 100 and less than 1 
ages_check = len(df[(df['Age'] < 1) | (df['Age'] > 100)])
ages_check

0

##### Insights
--------------
- We have no real outliers in the Age column


### 8. Credit Score Data Integrity Check

- the credit score typically ranges from 300 to 900

In [11]:
# Filter the DataFrame for values in the "Age" column greater than 900 and less than 300
credit_check = len(df[(df['CreditScore'] < 300) | (df['CreditScore'] > 900)])
credit_check

0

##### Insight:
------------
- There are no real outliers in the credit score column either

### 9. Converting Estimated Salary and Balance to Integer Data Type

In [12]:
df['EstimatedSalary'] = df['EstimatedSalary'].astype(int)
df['Balance'] = df['Balance'].astype(int)

df['EstimatedSalary'].dtype, df['Balance'].dtype

(dtype('int32'), dtype('int32'))

### 10. Label Encoding for Categorical Data

In [13]:
le = LabelEncoder()
lstforle = ['Geography', 'Gender']
for col in lstforle:
    df[col] = le.fit_transform(df[col])


[df['Gender'].unique(), 
df['Geography'].unique()] 

[array([0, 1]), array([0, 2, 1])]

### 11. Percentage of Churned Customers based on Credit Card Ownership

- check how the column (HasCrCard) relates to the 'Exited' column. This analysis helps us understand how credit card services affect customer retention strategies

In [14]:
credit_churn_percentage = df.groupby('HasCrCard')['Exited'].mean()*100
credit_churn_percentage

HasCrCard
0    20.814941
1    20.184266
Name: Exited, dtype: float64

##### Insights:
--------------
- credit card services have no impact on customer retention

### 12. Removing Credit Card Ownership Column

In [15]:
df = df.drop('HasCrCard', axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0,1,1,101348,1
1,608,2,0,41,1,83807,1,1,112542,0
2,502,0,0,42,8,159660,3,0,113931,1
3,699,0,0,39,1,0,2,0,93826,0
4,850,2,0,43,2,125510,1,1,79084,0
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0,2,0,96270,0
9996,516,0,1,35,10,57369,1,1,101699,0
9997,709,0,0,36,7,0,1,1,42085,1
9998,772,1,1,42,3,75075,2,0,92888,1


___________

## Visually verify results 

In [16]:
i = df.sample(1).index[0]
print(raw_df['Geography'].iloc[i])
print(df['Geography'].iloc[i])

Germany
1


In [17]:
i = df.sample(1).index[0]
print(raw_df['Gender'].iloc[i])
print(df['Gender'].iloc[i])

Female
0


# Export Data 

In [18]:
df.to_pickle(EXPORT_PATH)

---

In [20]:
pd.read_pickle(EXPORT_PATH)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0,1,1,101348,1
1,608,2,0,41,1,83807,1,1,112542,0
2,502,0,0,42,8,159660,3,0,113931,1
3,699,0,0,39,1,0,2,0,93826,0
4,850,2,0,43,2,125510,1,1,79084,0
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0,2,0,96270,0
9996,516,0,1,35,10,57369,1,1,101699,0
9997,709,0,0,36,7,0,1,1,42085,1
9998,772,1,1,42,3,75075,2,0,92888,1


In [23]:
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [24]:
df.dtypes

CreditScore        int64
Geography          int32
Gender             int32
Age                int64
Tenure             int64
Balance            int32
NumOfProducts      int64
IsActiveMember     int64
EstimatedSalary    int32
Exited             int64
dtype: object